# Dash

### Dash en 20 minutos

In [80]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

app = Dash() #Inicializa la app

# Convert DataFrame values to native Python types
df_native = df.copy()
for col in df_native.select_dtypes(include=['float', 'int']).columns:
    df_native[col] = df_native[col].apply(lambda x: float(x) if pd.notnull(x) else None)

app.layout = html.Div([
    html.Div(children='My First App with Data, Graph, and Controls'), #Abres un div de html
    html.Hr(), # Línea horizontal para separar secciones
    dcc.RadioItems(options=['pop', 'lifeExp', 'gdpPercap'], value='lifeExp', id='my-final-radio-item-example'), #Botones de radio para seleccionar la columna, lifeExp es el valor por defecto
    dash_table.DataTable(
        data=[
            {str(k): (v if isinstance(v, (str, float, int, bool)) else str(v)) for k, v in row.items()}
            for row in df_native.to_dict('records')
        ],
        page_size=6
    ), # Tabla de datos que muestra el DataFrame, con un tamaño de página de 6 filas
    dcc.Graph(figure={}, id='my-final-graph-example')# Gráfico vacío que se actualizará con la interacción del usuario
])

@callback(
    Output(component_id='my-final-graph-example', component_property='figure'), # Actualiza la figura del gráfico
    Input(component_id='my-final-radio-item-example', component_property='value') # Toma el valor seleccionado del botón de radio como entrada
)
#Funcion del callback que actualiza el gráfico, es decir, cuando el usuario selecciona un valor del botón de radio, se llama a esta función
def update_graph(col_chosen):
    fig = px.histogram(df, x='continent', y=col_chosen, histfunc='avg') # Crea un histograma agrupado por continente, mostrando el promedio de la columna seleccionada
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True) # Ejecuta la aplicación en modo de depuración

# Fundamentos de Dash

### Layout

### Callbacks basicos

In [81]:
app.layout = html.Div([
    html.H1(id='titulo-pagina', children='Selecciona una opción'),
    dcc.Dropdown(
        id='dropdown-opciones', #* Aqui seleccionamos un valor
        options=[
            {'label': 'Buenos días', 'value': 'Hola Mundo'},
            {'label': 'Buenas tardes', 'value': 'Hola Dash'}  #* ///////////    Contenido principal de la pagina      /////////////
        ],
        value='Hola Mundo'
    )
])

# Definimos el callback
@app.callback(
    Output('titulo-pagina', 'children'), # <-- Esta es la Salida (Output) #* La salida que tendra se mostrara en el elemento que tenga el id, en este caso se mostrara como H1, asi igual con graficos y asi *
    Input('dropdown-opciones', 'value')  # <-- Esta es la Entrada (Input) #* Aqui ponemos el id o classname de el elemento que queremos que dispare el callback y value es lo que vale el Dropdown
)
def actualizar_titulo(valor_seleccionado):
    """
    Esta es la función del callback.
    Recibe el valor del dropdown y lo devuelve para actualizar el H1.
    """
    return valor_seleccionado

if __name__ == '__main__':
    app.run(jupyter_height=200)  # Ejecuta la aplicación en modo Jupyter con una altura de 200 píxeles

### Graficos interactivos y filtrado cruzado

### Compartir datos entre devoluciones de llamada

# Devoluciones de llamada de dash

##### Callbacks avanzados

##### - PreventUpdate

Utilizamos PreventUpdate de la libreria de Dash .exceptions para hacer que nuestro Input no actualice la salida del callback, es decir solamente muestra un vez la salida.

In [82]:
from dash.exceptions import PreventUpdate

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.Button('Click here to see the content', id='show-secret'),
    html.Div(id='body-div')
])

@callback(
    Output('body-div', 'children'),
    Input('show-secret', 'n_clicks')
)
def update_output(n_clicks):
    if n_clicks is None:
        raise PreventUpdate
    else:
        return "Elephants are the only animal that can't jump"


app.run(jupyter_height=200)  # Ejecuta la aplicación en modo Jupyter con una altura de 200 píxeles


##### - no_update

En Dash, el objeto no_update se utiliza dentro de los callbacks para indicar que una salida específica no debe actualizarse, incluso si el callback se ejecuta. Es útil cuando tienes múltiples salidas en un callback y quieres que solo algunas se actualicen, mientras que otras permanezcan igual.

In [ ]:
import dash
from dash import html, dcc, Input, Output, no_update

app.layout = html.Div([
    dcc.Input(id="input-text", type="text", placeholder="Escribe algo"),
    html.Button("Actualizar", id="btn", n_clicks=0),
    html.Div(id="output-1"),
    html.Div(id="output-2")
])

@app.callback(
    [Output("output-1", "children"),
     Output("output-2", "children")],
    Input("btn", "n_clicks"),
    Input("input-text", "value"),
    prevent_initial_call=True
)
def update_divs(n_clicks, text):
    if not text:  
        # Si no hay texto, solo actualizo el output-1
        return f"Clicks: {n_clicks}", no_update
    else:
        # Si hay texto, actualizo los dos
        return f"Clicks: {n_clicks}", f"Texto: {text}"

app.run(jupyter_height=200)  # Ejecuta la aplicación en modo Jupyter con una altura de 200 píxeles


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File c:\Users\yahir\anaconda3\Lib\site-packages\dash\dash.py:1378, in Dash._prepare_callback(
    self=<dash.dash.Dash object>,
    g={'inputs_list': [{'id': 'setprops-row-selection-...n': 'http://127.0.0.1:8050', 'updated_props': {}},
    body={'changedPropIds': ['setprops-row-selection-popup.selectedRows'], 'inputs': [{'id': 'setprops-row-selection-popup', 'property': 'selectedRows', 'value': []}, {'id': 'setprops-row-selection-modal-close', 'property': 'n_clicks', 'value': 2}], 'output': 'a06acae8e024a993eb57d46c8103f08265cb23e26b548da39c9ce0a0f86fc3e1', 'parsedChangedPropsIds': ['setprops-row-selection-popup.selectedRows']}
)
   1377 try:
-> 1378     cb = self.callback_map[output]
        output = 'a06acae8e024a993eb57d46c8103f08265cb23e26b548da39c9ce0a0f86fc3e1'
        self.callback_map = {'..out.children...err.children..': {'inpu

Basicamente aqui lo que hace el codigo es que cuando presionamos el boton, si no hay nada escrito solo se actualiza la salida de los clicks y el `no_update` hace que no se modifique su salida, en cambio la del texto no, ya si se cambia el texto pues ya se cambian las dos salidas

##### - runnnig and State

En Dash, `State` se utiliza para obtener el valor actual de un componente sin que dispare el callback. Es útil cuando quieres usar el valor de un input, pero el callback se activa por otro componente (por ejemplo, un botón).


El parámetro `running` en los callbacks permite saber si el callback está en ejecución, lo que es útil para mostrar indicadores de carga o bloquear acciones mientras se procesa.

`running` acepta una lista de tres tuplas, donde cada elemento de la tupla tiene:

- El **primero** es el output, es el componente que se quiere modificar mientras el callback se esta ejecutando, junto con el output se pone una caracteristica o un estilo, puede ser un estado como deshabilitado, estilo o un componente hijo que contenga texto

- El **segundo** es el valor cuando se esta corriendo, aqui se caracteristica que viene con el output le asignamos un valor, puede ser True, un conjunto de estilos o texto.

- El **tercer** valor es el valor que obtiene esta carcateritica cuando se termina de correr.

In [44]:
from dash import Dash, html, Output, Input, State, dcc
app = Dash(__name__)
app.layout = html.Div([
    dcc.Input(id='input-valor', type='text', value=''),
    html.Button('Mostrar valor', id='btn-mostrar'),
    html.Div(id='output-valor')
])

@app.callback(
    Output('output-valor', 'children'),
    Input('btn-mostrar', 'n_clicks'),
    State('input-valor', 'value'), # Estado del input
    running=[(Output('output-valor', 'disabled'), True, False) ]  # Esto deshabilita el botón mientras se procesa el callback
)
def mostrar_valor(n_clicks, valor):
    if n_clicks is None:
        return '', ''
    import time
    time.sleep(2)  # Simula proceso largo
    return f'El valor actual es: {valor}', 'Carga finalizada'

app.run(jupyter_height=100)

##### - callback_context

Con dash.callback_context puedes ver los metadatos de la ejecucion de un callback.

callback_context tiene varias propiedades como:

- **triggered_id** -> Es el id del componente que activo la devolucion de la llamada

- **triggered_prop_ids** -> Muestra un diccionario de los identificadores y propiedades de los componentes de la devolucion de llamada, util cuando varios Inputs pueden activar un solo output.

- **args_grouping** -> Este es otro diccionario que muestra las entrada utlizadas con firmas de devolucion de llamada flexible (se vera mas adelnate de esto). Este diccionario contiene:

-------------------------

* ***"id"***: ID del componente
* ***"id_str***: Para los identificadores de coincidencia de patrones, es el identificador del diccionario convertido en cadena sin espacios en blanco
* ***value***: el valor de la propiedad del componente utilizada en la devolucion de llamada * ***activado***: un valor booleano que indica si esta entrada se activo la devolucion de llamada

--------------------------

- **triggered** -> Lista de todas las propiedades de los Input que cambiaron y provocaron la ejecucion de la devolucion de llamada

- **inputs y states** -> Son los parametros de devolucion de llamada con el ID y su propiedad

- **outputs_list, inputs_list y states_list** -> Listas de entradas, salidas y elementos de estado, organizado como se encuentran en los argumentos de la devolucion de llamada y el valor de retorno.

- ***response** -> el objeto de respuesta HTTP

- **record_timing** -> Un metodo para informacion de tiempo granular

In [45]:
from dash import Dash, html, Output, Input, State, callback_context
app = Dash(__name__)
app.layout = html.Div([
    html.Button('Botón 1', id='btn-1'),
    html.Button('Botón 2', id='btn-2'),
    dcc.Input(id='input-1', type='text', value=''),
    html.Div(id='output-1'),
    html.Div(id='output-ctx')
])

@app.callback(
    Output('output-1', 'children'),
    Output('output-ctx', 'children'),
    Input('btn-1', 'n_clicks'),
    Input('btn-2', 'n_clicks'),
    State('input-1', 'value')
)
def ejemplo_callback(n1, n2, valor):
    ctx = callback_context
    # triggered: lista de inputs que activaron el callback
    triggered = ctx.triggered
    # triggered_id: id del componente que disparó el callback
    triggered_id = ctx.triggered[0]['prop_id'].split('.')[0] if triggered else None
    # inputs y states: diccionarios de valores actuales
    inputs = ctx.inputs
    states = ctx.states
    # outputs_list: lista de outputs
    outputs_list = ctx.outputs_list
    # args_grouping: agrupación avanzada de argumentos
    args_grouping = ctx.args_grouping
    info = f'Componente activado: {triggered_id}\n'
    info += f'Triggered: {triggered}\n'
    info += f'Inputs: {inputs}\n'
    info += f'States: {states}\n'
    info += f'Outputs: {outputs_list}\n'
    info += f'Args grouping: {args_grouping}'
    return f'Valor del input: {valor}', info

app.run(jupyter_height=200)

##### - Memorizacion

La memorizacion permite que cuando un proceso requiera hacer calculos largos y complejos, pueda almacenar el resultado. De forma que al llamar de nuevo la funcion no se demore tanto desde la primera vez que se mando a llamar el callback

In [46]:
from functools import lru_cache # Hermonito, aqui ocupamos la libreria de lru_cache
import time
app.layout = html.Div([
    html.H3('Calcula el cuadrado (con memorización)'),
    dcc.Input(id='input-num', type='number', value=2),
    html.Button('Calcular', id='btn-calc'),
    html.Div(id='output-mem')
])

@lru_cache(maxsize=32)
def operacion_costosa(x):
    time.sleep(2)  # Simula proceso lento
    return x ** 2

@app.callback(
    Output('output-mem', 'children'),
    Input('btn-calc', 'n_clicks'),
    State('input-num', 'value')
)
def calcular(n_clicks, valor):
    if n_clicks is None or valor is None:
        return 'Introduce un número y presiona Calcular.'
    resultado = operacion_costosa(valor)
    return f'El cuadrado de {valor} es {resultado} (calculado con memorización)'

app.run(jupyter_height=120)

Basicamente cuando tu pones un numero presionas calcular, si lo vuelves hacer con el mismo numero se te cargara el resultado mucho mas rapido debido a la memorizacion.

##### - allow_optional

`allow_optional` se utiliza en los callback, normalmente en los Input, que indican que una entrada es opcional, aqui lo que pasa es que cuando tenemos un componente que no esta en el layout de la pagina, dash nos dara error, en cambio si ponemos `allow_optional` se volvera `None`. Es bastante util cuando tenemos componentes que pueden aparecer o no dependiendo de la interaccion del usuario, por estar en un condicional por ejemplo.

In [47]:
from dash import Dash, html, Output, Input, State, no_update

app = Dash(suppress_callback_exceptions=True)

app.layout = html.Div([
    html.Div([
        html.Button(id="optional-inputs-button-1", children="Button 1", className="button"),
        html.Div(id="optional-inputs-container"),
        html.Div(id="optional-inputs-output", className="output")
    ], className="container")
])

@app.callback(
    Output("optional-inputs-container", "children"),
    Input("optional-inputs-button-1", "n_clicks"),
    State("optional-inputs-container", "children"),
    prevent_initial_call=True
)
def add_second_button(_, current_children):
    if not current_children:
        return html.Button(id="optional-inputs-button-2", children="Button 2", className="button")
    return no_update

@app.callback(
    Output("optional-inputs-output", "children"),
    Input("optional-inputs-button-1", "n_clicks"),
    Input("optional-inputs-button-2", "n_clicks", allow_optional=True),
    prevent_initial_call=True
)
def display_clicks(n_clicks1, n_clicks2):
    return f"Button 1 clicks: {n_clicks1} - Button 2 clicks: {n_clicks2}"

app.run(jupyter_height=100)


##### - Async/Await

El async/await nos ayuda a realizar funciones asincronas en los callback. Aunque es propia de Python y no de dash, resulta util cuando queremos hacer peticiones a una base de datos por ejemplo.

In [48]:
import asyncio

# Función asíncrona simulando un request lento
async def fetch_data(seconds: int):
    await asyncio.sleep(seconds)
    return f"Datos obtenidos después de {seconds} segundos"

app.layout = html.Div([
    html.H1("Ejemplo de async/await en Dash"),
    dcc.Input(id="input-seconds", type="number", value=2, min=1, max=10),
    html.Button("Obtener datos", id="btn-fetch", n_clicks=0),
    html.Div(id="output-data")
])

@app.callback(
    Output("output-data", "children"),
    Input("btn-fetch", "n_clicks"),
    Input("input-seconds", "value"),
    prevent_initial_call=True
)
def update_output(n_clicks, seconds):
    # Ejecutar la función asíncrona dentro del callback
    result = asyncio.run(fetch_data(seconds))
    return result

app.run(jupyter_height=150)

##### - set_props

En Dash, set_props no es algo que uses tú directamente en la mayoría de los casos, sino que es una función interna que Dash expone en el frontend (JavaScript) y que se encarga de actualizar las propiedades de un componente desde el backend (Python).

Cada componente de Dash (ejemplo: dcc.Input, html.Div, etc.) tiene props (propiedades).

* Ejemplo: un dcc.Input tiene value, placeholder, type, etc.

Cuando tú usas un callback, el servidor (Python) devuelve un valor que debe asignarse a alguna propiedad de un componente en el cliente (el navegador).

Dash en el navegador recibe ese valor y usa set_props para aplicarlo dinámicamente al componente correspondiente.

In [76]:
import dash_ag_grid as dag
from dash import Dash, Input, html, ctx, callback, set_props
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.SPACELAB])

rowData = [ # Hacemos un arreglo de datos que indican precios de modelos de coches
    {"make": "Toyota", "model": "Celica", "price": 35000},
    {"make": "Ford", "model": "Mondeo", "price": 32000},
    {"make": "Porsche", "model": "Boxster", "price": 72000},
]

app.layout = html.Div(
    [
        dag.AgGrid( # Aqui agregamos un grid para mostrar los datos
            id="setprops-row-selection-popup", # Le ponemos el id que es el que usaremos para el set_props
            rowData=rowData,
            columnDefs=[{"field": i} for i in ["make", "model", "price"]], # Definimos las columnas que se mostrarán en el grid
            columnSize="sizeToFit", # Ajustamos el tamaño de las columnas
            dashGridOptions={"rowSelection": {'mode': 'single'}, "animateRows": False}, # Ponemos como opciones para la selección de filas primero que tenga el modo single, este modo es para seleccionar una sola fila y animateRows es para desactivar la animación de las filas
        ),
        dbc.Modal(
            [
                dbc.ModalHeader("More information about selected row"),
                dbc.ModalBody(id="setprops-row-selection-modal-content"),
                dbc.ModalFooter(dbc.Button("Close", id="setprops-row-selection-modal-close", className="ml-auto")),
            ],
            id="setprops-row-selection-modal",
        ),
    ]
)

@callback(
    Input("setprops-row-selection-popup", "selectedRows"),
    Input("setprops-row-selection-modal-close", "n_clicks"),
    prevent_initial_call=True, # Evitamos que se dispare la llamada al inicio
)
def open_modal(selection, _):
    if ctx.triggered_id == "setprops-row-selection-modal-close": #Aqui decimos que si suena el boton de cerrar
        # Close the modal
        set_props("setprops-row-selection-modal", {'is_open': False}) #Entonces cerramos el modal
    elif ctx.triggered_id == "setprops-row-selection-popup" and selection: #Aqui decimos que si suena el grid y hay una seleccion
        # Open the modal and display the selected row content
        content_to_display = "You selected " + ", ".join( # Aqui mostramos la seleccion
            [
                f"{s['make']} (model {s['model']} and price {s['price']})"
                for s in selection
            ]
        )
        set_props("setprops-row-selection-modal", {'is_open': True}) #Entonces abrimos el modal
        set_props("setprops-row-selection-modal-content", {'children': content_to_display}) #Aqui mostramos el contenido


app.run(jupyter_height=400)

Basicamente lo que hace el codigo es que muestra una ventana con la celda seleccionada cuando el usuario selecciona un fila en el grid, y lo cierra cuando da el boton de cerrar. Utiliza set_props para actualizar las propiedades del modal y su contenido

##### - callbacks circulares

En los callbacks circulares, basicamente tu puedes hacer que se pueda modificar una entrada para tener una salida y despues tu puedes modificar esa salida para que se modifique esa entrada. Ejemplo:

In [ ]:
from dash import Dash, html, dcc, Input, Output, callback, ctx

external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]


app.layout = html.Div([
    html.Div('Convert Temperature'),
    'Celsius',
    dcc.Input(
        id="celsius",
        value=0.0,
        type="number"
    ),
    ' = Fahrenheit',
    dcc.Input(
        id="fahrenheit",
        value=32.0,
        type="number",
    ),
])

@callback(
    Output("celsius", "value"),
    Output("fahrenheit", "value"),
    Input("celsius", "value"),
    Input("fahrenheit", "value"),
)
def sync_input(celsius, fahrenheit):
    input_id = ctx.triggered[0]["prop_id"].split(".")[0] # Esta propiedad es muy utilizada, basicamente nos dice que input fue activado, esto nos ayuda a saber que conversion hacer
    if input_id == "celsius":
        fahrenheit= None if celsius is None else (float(celsius) * 9/5) + 32
    else:
        celsius = None if fahrenheit is None else (float(fahrenheit) - 32) * 5/9
    return celsius, fahrenheit

app.run(jupyter_height=100)


##### Callbacks del lado del cliente

A veces las devoluciones de llamada pueden generar una sobrecarga significativa, cuando se reciben y devuelven cantidades muy grandes de datos, se llaman muy a menudo, son parte de una cadena de devolucion de llamada que requeire multiples viajes de ida y vuelta entre el navegador y Dash

Es por ello que el costo general de una devolucion de llamada se vuelve muy grande y no es posible ninguna otra optimizacion. Por ello se opta por escribir las llamda utilizando JavaScript a partir de `clientside_dash` y `ClientsideFunction`.

Estas funciones, funcionan, con un archivo externo de JavaScript que va en la carpeta de `/assets`, dash lo detecta y sirve automaticamente estos archivos poniendose a disposicion de la aplicacion de Dash

In [ ]:
#=========== Este es el codigo que se utiliza para Dash ============

import dash
from dash import dcc, html
from dash.dependencies import Input, Output, ClientsideFunction
import plotly.graph_objs as go

app = dash.Dash(__name__)
app.layout = html.Div([
    dcc.Slider(id='slider', min=0, max=10, step=1, value=5),
    dcc.Graph(id='graph',
              figure=go.Figure(go.Scatter(x=[0,1,2,3,4,5], y=[0,1,4,9,16,25])))
])
# Callback que se ejecuta en el navegador usando JavaScript
app.clientside_callback( # Aqui se define el callback del lado del cliente
    ClientsideFunction(namespace='clientside', function_name='update_graph'), # Aqui se llama a la funcion de JavaScript
    Output('graph', 'figure'),
    Input('slider', 'value')
)

app.run(jupyter_height=300)  # Ejecuta la aplicación en modo Jupyter con una altura de 300 píxeles

Esta es toda la funcion que va en el archivo `/assets` que nos permitira modificar el grafico dinamicamente

In [ ]:
# window.clientside = {}; # Espacio de nombres para funciones del lado del cliente (namespace)
# window.clientside.update_graph = function(value) { # Función para actualizar el gráfico
#     #// Genera datos dinámicamente usando el valor del slider
#     var x = [];
#     var y = [];
#     for (var i = 0; i <= value; i++) {
#         x.push(i);
#         y.push(i * i);
#     }
#     return {
#         'data': [{
#             'x': x,
#             'y': y,
#             'type': 'scatter',
#             'mode': 'lines+markers',
#             'marker': {'color': 'red'}
#         }],
#         'layout': {
#             'title': 'Gráfico actualizado con JavaScript',
#             'xaxis': {'title': 'X'},
#             'yaxis': {'title': 'Y = X^2'}
#         }
#     };
# };


Otra forma de poner la funcion de JavaScript sin hacer un archivo a parte es la siguiente:

In [ ]:
# =========== La misma funcion pero en Dash =================

app.clientside_callback(
    """
        function update_graph(value) {
            var x = [];
            var y = [];
            for (var i = 0; i <= value; i++) {
                x.push(i);
                y.push(i * i);
            }
            return {
                'data': [{
                    'x': x,
                    'y': y,
                    'type': 'scatter',
                    'mode': 'lines+markers',
                    'marker': {'color': 'red'}
                }],
                'layout': {
                    'title': 'Gráfico actualizado con JavaScript',
                    'xaxis': {'title': 'X'},
                    'yaxis': {'title': 'Y = X^2'}
                }
            };
        }
    """,
    Output('graph', 'figure'),
    Input('slider', 'value')
)


Para hacer funciones mas cortas sirve la ultima opcion, en cambio si son mas grandes es mas organizado la primer forma.

##### Store

Utilizar `Store` es bastante util en Dash, te permite guardar datos en el navegador ya sea en memoria (memmory) se borra al cargar la pagina, local te lo deja almacenado aunque cierres el navegador y session que se almacena hasta que se cierra la sesion del navegador.

In [1]:
import dash
from dash import html, dcc, Input, Output, State

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Input(id="input-text", type="text", placeholder="Escribe algo..."), # Campo de texto
    html.Button("Guardar en Store", id="btn-save"), # Botón para guardar
    html.Hr(), # Línea horizontal

    dcc.Store(id="memory-store", storage_type="memory"),  # opciones: memory, local, session, aqui le decimos que guardamos en memoria

    html.Button("Mostrar desde Store", id="btn-show"), # Botón para mostrar
    html.Div(id="output-text") # Div para mostrar el texto
])

# Callback del lado del cliente para guardar en Store
app.clientside_callback(
    """
    function(n_clicks, value, data) { 
        if (!n_clicks) {
            return data || {};  // no sobreescribir si no se presionó
        }
        return { "mensaje": value };
    }
    """,
    Output("memory-store", "data"),
    Input("btn-save", "n_clicks"),
    State("input-text", "value"),
    State("memory-store", "data")
)

# Callback del lado del cliente para leer del Store
app.clientside_callback(
    """
    function(n_clicks, data) {
        if (!n_clicks || !data) {
            return "⚠️ Nada guardado aún";
        }
        return "Dato guardado: " + data.mensaje;
    }
    """,
    Output("output-text", "children"),
    Input("btn-show", "n_clicks"),
    State("memory-store", "data")
)

app.run(jupyter_height=200)  # Ejecuta la aplicación en modo Jupyter con una altura de 200 píxeles


##### Devoluciones de llamada de coincidencia de patrones

##### - ALL

##### - MATCH

##### - ALL SMALLER

##### Actualizaciones parciales de la propiedad

##### - Patch

##### Devoluciones de llamada en segundo plano

##### Firmas de devolucion de llamada flexibles

##### Salidas de devolucion de llamada duplicadas

##### Como determinar que entrada de devolucion de llamada cambio

##### Manejadores de errores de devolucion de llamada

##### Problemas con las devoluciones de llamada

# Biblioteca de componentes de codigo abierto

### Componentes principales de Dash (dcc)

In [50]:
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash

app = JupyterDash(__name__)

c:\Users\yahir\anaconda3\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



##### Markdown

In [51]:
app.layout = html.Div([
    dcc.Markdown('''
        ### Título del Gráfico
        Este es un gráfico de ejemplo utilizando Dash y Plotly.
    ''', 
    mathjax=True,
    # children=[
    #     #* Aqui se puede agregar más contenido, como otros gráficos o elementos de texto
    # ]
    ),
])

app.run(jupyter_height=200)

##### Dropdown

In [52]:
app.layout = html.Div([
    dcc.Dropdown(
        options=[
            {'label': 'Opción 1', 'value': 'opcion1'},
            {'label': 'Opción 2', 'value': 'opcion2'},
            {'label': 'Opción 3', 'value': 'opcion3'}
        ],
        value='opcion1',  # Valor por defecto
        id='my-dropdown-example',  # ID del dropdown para poder referenciarlo en callbacks
        multi=False,  # Permite seleccionar múltiples opciones si se establece en True
        clearable=True,  # Permite limpiar la selección
        placeholder='Selecciona una opción'  # Texto que aparece cuando no hay ninguna opción seleccionada
    )
])

app.run(jupyter_height=200)

##### Slider

In [53]:
app.layout = html.Div([
    dcc.Slider(
        min=0,
        max=100,
        step=1, # Paso del slider
        value=50, # Valor inicial del slider
        marks={i: str(i) for i in range(0, 101, 10)}, # Marcas del slider
        id='my-slider-example'  # ID del slider para poder referenciarlo en callbacks
    )
])
app.run(jupyter_height=200)


##### DatepickleSingle and Range

In [54]:
app.layout = html.Div([
    html.H3('Selecciona una fecha:'),
    dcc.DatePickerSingle(
        id='my-date-picker',
        date='2023-01-01'
    ),
    html.Hr(),
    html.H3('Selecciona un rango de fechas:'),
    dcc.DatePickerRange(
        id='my-date-picker-range',
        start_date='2023-01-01',
        end_date='2023-12-31'
    )
])

app.run(jupyter_height=200)


##### Tab

In [55]:
app.layout = html.Div([
    dcc.Tab(
        label='Tab 1',
        value='tab-1',
        children=[
            html.Div('Contenido del Tab 1')
        ]
    ),
    dcc.Tab(
        label='Tab 2',
        value='tab-2',
        children=[
            html.Div('Contenido del Tab 2')
        ]
    ),
    dcc.Tab(
        label='Tab 3',
        value='tab-3',
        children=[
            html.Div('Contenido del Tab 3')
        ]
    )
])

app.run(jupyter_height=200)

##### Tabs

In [56]:
app.layout = html.Div([
    dcc.Tabs(
        id='my-tabs-example',  # ID de las pestañas para poder referenciarlas en callbacks
        value='tab-1',  # Valor por defecto de la pestaña activa
        children=[
            dcc.Tab(label='Pestaña 1', value='tab-1'),
            dcc.Tab(label='Pestaña 2', value='tab-2'),
            dcc.Tab(label='Pestaña 3', value='tab-3')
        ]
    )
])
app.run(jupyter_height=200)

##### Checklist

In [57]:
app.layout = html.Div([
    dcc.Checklist(
        options=[
            {'label': 'Opción 1', 'value': 'opcion-1'},
            {'label': 'Opción 2', 'value': 'opcion-2'},
            {'label': 'Opción 3', 'value': 'opcion-3'}
        ],
        value=['opcion-1', 'opcion-2'],  # Valores seleccionados por defecto
        id='my-checklist',
        labelStyle={'display': 'block'},  # Estilo de las etiquetas para que se muestren en bloque
        inline=True
    )
])
app.run(jupyter_height=200)

##### RadioItems

In [58]:
app.layout = html.Div([
    dcc.RadioItems(
        options=[
            {'label': 'Opción 1', 'value': 'opcion-1'},
            {'label': 'Opción 2', 'value': 'opcion-2'},
            {'label': 'Opción 3', 'value': 'opcion-3'}
        ],
        value='opcion-1',  # Valor seleccionado por defecto
        id='my-radio-items',
        labelStyle={'display': 'block'},  # Estilo de las etiquetas para que se muestren en bloque
        inline=True
    )
])
app.run(jupyter_height=200)

##### RangeSlider

In [59]:
app.layout = html.Div([
    dcc.Input(
        type='text',  # Tipo de entrada, puede ser 'text', 'number', 'password', etc.
        id='my-input',
        value='Texto por defecto',  # Valor por defecto
        placeholder='Introduce un texto',
        debounce=True,  # Habilita el debounce para evitar múltiples actualizaciones al escribir
    )
])
app.run(jupyter_height=200)

##### Textarea

In [60]:
app.layout = html.Div([
    dcc.Textarea(
        value='Texto por defecto',
        placeholder='Introduce un texto',
        style={'width': '100%', 'height': 200},  # Estilo para el área de texto
    )
])
app.run(jupyter_height=200)

##### Download

In [61]:
app.layout = html.Div([
    dcc.Download(
        id="download-data",
        data=dict( # Datos a descargar
            content="Texto de ejemplo para descargar",
            filename="ejemplo.txt"
        ),
        type='text/plain'  # Tipo de contenido, puede ser 'text/plain', 'application/json', etc.
    )
])
app.run(jupyter_height=200)

##### Loading

In [62]:
from dash import Output, Input
import time

app.layout = html.Div([
    dcc.Loading(  # Componente de carga para mostrar un indicador de carga
        children=[ # Sobre estos componentes es donde se mostrará el indicador de carga
            dcc.Input(
                type='text',
                id='my-input',
                value='Texto por defecto',
                placeholder='Introduce un texto',
                debounce=True,
            ),
            dcc.Textarea(
                value='Texto por defecto',
                placeholder='Introduce un texto',
                style={'width': '100%', 'height': 200},
            ),
            dcc.Clipboard(),
            dcc.Download(
                id="download-data",
                data=dict(
                    content="Texto de ejemplo para descargar",
                    filename="ejemplo.txt"
                ),
                type='text/plain'
            )
        ],
        type='circle',  # Tipo de indicador de carga, puede ser 'circle', 'default', 'dot', etc.
        fullscreen=True,  # Habilita el modo de pantalla completa
        debug=True,
        delay_show=100,  # Retraso en milisegundos para mostrar el indicador de carga
        delay_hide=200,  # Retraso en milisegundos para ocultar el indicador de carga
        show_initially=True,
        overlay_style={'backgroundColor': 'rgba(255, 255, 255, 0.8)'},
        style={'width': '100%', 'height': '100%'},  # Estilo para ocupar toda la pantalla
        className='my-loading-indicator',
        custom_spinner=dcc.Loading(
            type='circle',
            children=[
                dcc.Input(
                    type='text',
                    id='my-input',
                    value='Texto por defecto',
                    placeholder='Introduce un texto',
                    debounce=True,
                ),
                dcc.Textarea(
                    value='Texto por defecto',
                    placeholder='Introduce un texto',
                    style={'width': '100%', 'height': 200},
                ),
                dcc.Clipboard(),
                dcc.Download(
                    id="download-data",
                    data=dict(
                        content="Texto de ejemplo para descargar",
                        filename="ejemplo.txt"
                    ),
                    type='text/plain'
                )
            ]
        )
    )
])

@app.callback(
    Output('my-input', 'value'),
    Input('my-input', 'value')
)
def update_input(n_clicks):
    if n_clicks:
        time.sleep(3)  # Simula una operación que toma tiempo
        return f"Texto actualizado después de {n_clicks} clics"
    return ""
app.run(jupyter_height=200)

##### Store

In [63]:
app.layout = html.Div([
    dcc.Store(
        id='my-store',  # ID del componente Store para poder referenciarlo en callbacks
        data=dict(
            my_input_value='Texto por defecto',
            my_textarea_value='Texto por defecto'
        ),  # Datos iniciales del componente Store
        storage_type='local',  # Tipo de almacenamiento, puede ser 'local', 'session' o 'memory'
        clear_data=True,  # Limpia los datos al recargar la página
    )
])
app.run(jupyter_height=200)

##### Tooltip

In [64]:
app.layout = html.Div([
    dcc.Tooltip(
        children=[
            html.Button("Hover over me", id="tooltip-target"),
            dcc.Tooltip(
                id="tooltip",
                children="This is a tooltip"
            )
        ],
        id="my-tooltip-example",  # ID del tooltip para poder referenciarlo en callbacks
        show=True,
        bbox=dict(
            x=100,
            y=100,
            width=200,
            height=100
        ),
        direction="top",
        background_color="lightblue",
        style={'fontSize': 16, 'padding': '10px'},  # Estilo del tooltip
        border_color="blue",
        loading_text="Cargando..."
    )
])
app.run(jupyter_height=200)

### Componentes HTML de Dash

##### Estructura y contenedores

In [65]:
app.layout = html.Div([
    html.Div([
        "Div"
    ], style={'border': '1px solid black', 'padding': '10px'}),
    html.Section([
        "Section"
    ], style={'border': '1px solid red', 'padding': '10px'}   ),
    html.Article([
        "Article"
    ], style={'border': '1px solid green', 'padding': '10px'}),
    html.Aside([
        "Aside"
    ], style={'border': '1px solid blue', 'padding': '10px'}),
    html.Header([
        "Header"
    ], style={'border': '1px solid purple', 'padding': '10px'}),
    html.Main([
        "Main"
    ], style={'border': '1px solid orange', 'padding': '10px'}),
    html.Footer([
        "Footer"
    ], style={'border': '1px solid pink', 'padding': '10px'})
])

app.run(jupyter_height=200)

##### Texto y Titulos

In [66]:
app.layout = html.Div([
    html.H1('Título Principal', style={'textAlign': 'center', 'color': 'blue'}),
    html.H2('Subtítulo', style={'textAlign': 'left', 'color': 'green'}),
    html.H3('Sub-subtítulo', style={'textAlign': 'right', 'color': 'red'}),
    html.H4('Sub-sub-subtítulo', style={'textAlign': 'center', 'color': 'purple'}),
    html.H5('Sub-sub-sub-subtítulo', style={'textAlign': 'left', 'color': 'orange'}),
    html.H6('Sub-sub-sub-sub-subtítulo', style={'textAlign': 'right', 'color': 'brown'}),
    html.P('Este es un párrafo de ejemplo. Puedes agregar más texto aquí para describir el contenido de tu aplicación.'),
    html.Span('Texto en un span', style={'color': 'gray'}),
    html.Br(),  # Salto de línea
])

app.run(jupyter_height=200)

##### Listas

In [67]:
app.layout = html.Div([
    #* Lista ordenada*
    html.Ol([
        html.Li('Elemento 1'),
        html.Li('Elemento 2'),
        html.Li('Elemento 3')
    ], style={'listStyleType': 'decimal'}),
    #* Lista desordenada*
    html.Ul([
        html.Li('Elemento A'),
        html.Li('Elemento B'),
        html.Li('Elemento C')
    ], style={'listStyleType': 'disc'})
])

app.run(jupyter_height=200)

##### Enlaces e Imágenes

In [68]:
app.layout = html.Div([
    html.A('Este es un enlace', href='https://www.ejemplo.com', style={'color': 'blue'}),
    html.Br(),
    html.Img(src='https://th.bing.com/th/id/OIP.gem11rKpzU_-4NuRUeLJqQHaE7?w=242&h=180&c=7&r=0&o=5&dpr=1.5&pid=1.7', alt='Imagen de ejemplo', style={'width': '200px', 'height': 'auto'}),
    html.Br()
])
app.run(jupyter_height=200)

##### Formularios e interacción

In [69]:
app.layout = html.Div([
    html.Form([
        html.Button('Enviar', type='submit', style={'margin': '10px'}),
        html.Label('Nombre:', style={'margin-right': '10px'}),
        html.Select([
            html.Option('Opción 1', value='opcion1'),
            html.Option('Opción 2', value='opcion2'),
            html.Option('Opción 3', value='opcion3')
        ]),
        html.Optgroup(label='Grupo 1', children=[
            html.Option('Opción 4', value='opcion4'),
            html.Option('Opción 5', value='opcion5')
        ]),
        html.Select([
            html.Option('Opción 6', value='opcion6'),
            html.Option('Opción 7', value='opcion7'),
            html.Option('Opción 8', value='opcion8')
        ]),
        html.Textarea(name='mensaje', placeholder='Escribe tu mensaje aquí...', style={'width': '100%', 'height': '100px'})
    ])
])

app.run(jupyter_height=200)

##### Tablas

In [70]:
app.layout = html.Div([
    html.Table([
        html.Tr([ # Fila de encabezado
            html.Th('Nombre'), 
            html.Th('Opción Seleccionada'), # Columna de encabezado
            html.Th('Mensaje')
        ]),
        html.Tr([ # Fila de datos
            html.Td('Juan'),
            html.Td('Opción 1'), # columna de datos
            html.Td('Hola, este es un mensaje de prueba.')
        ]),

        #* ==== Estas son mas semanticas ======*

        html.Thead([ # Encabezado de la tabla
            html.Tr([
                html.Th('Nombre'),
                html.Th('Opción Seleccionada'),
                html.Th('Mensaje')
            ])
        ]),
        html.Tbody([
            html.Tr([
                html.Td('Juan'),
                html.Td('Opción 1'),
                html.Td('Hola, este es un mensaje de prueba.')
            ])
        ]),
        html.Tfoot([
            html.Tr([
                html.Td('Total'),
                html.Td('Opción 1'),
                html.Td('Hola, este es un mensaje de prueba.')
            ])
        ]),
        html.Colgroup([ # Agrupación de columnas, sirve para aplicar estilos a grupos de columnas
            html.Col(style={'width': '30%'}), # Columna 1
            html.Col(style={'width': '30%'}), # Columna 2
            html.Col(style={'width': '40%'}), # Columna 3
        ])
    ])
])
app.run(jupyter_height=200)

##### Media

In [71]:
app.layout = html.Div([
    html.Audio(src='ruta/a/tu/audio.mp3', controls=True),
    html.Video(src='ruta/a/tu/video.mp4', controls=True, style={'width': '100%', 'height': 'auto'}),
    html.Br(),
    html.Source(src='ruta/a/tu/archivo.mp3', type='audio/mpeg'), # Fuente de audio
    html.Track(src='ruta/a/tu/archivo.vtt', kind='subtitles', label='Español'), # Fuente de subtítulos
    html.Embed(src='ruta/a/tu/archivo.swf', type='application/x-shockwave-flash', width='100%', height='400px'), #Embed inserta archivos externos como PDF, videis, animaciones, etc
])
app.run(jupyter_height=200)

##### Elementos de formato de texto

In [72]:
app.layout = html.Div([
    html.B('Texto en negrita'),
    html.Br(),
    html.Strong('Texto en negrita y fuerte'),
    html.Br(),
    html.I('Texto en cursiva'),
    html.Br(),
    html.Em('Texto enfatizado'),
    html.Br(),
    html.Mark('Texto resaltado'),
    html.Br(),
    html.Small('Texto pequeño'),
    html.Br(),
    html.Del('Texto tachado'),
    html.Br(),
    html.Ins('Texto insertado'),
    html.Br(),
    html.Sub('Texto subíndice'),
    html.Br(),
    html.Sup('Texto superíndice'),
    html.Br(),
    html.Code('Texto en código'),
    html.Br(),
    html.Pre('Texto en preformateado'),
    html.Br(),
    html.Blockquote('Texto en bloque de cita'),
    html.Br(),
    html.Q('Texto en cita corta'),
])
app.run(jupyter_height=200)

##### Metadatos y script

In [73]:
app.layout = html.Div([
    html.Meta(name='viewport', content='width=device-width, initial-scale=1'),
    html.Title('Mi Aplicación Dash'),
    html.Link(rel='icon', type='image/x-icon', href='/assets/favicon.ico'),
    html.Div(id='contenido'),
    html.Script(src='/assets/script.js'),
    html.Link(rel='stylesheet', href='/assets/style.css'),  # Enlace a una hoja de estilos CSS
])
app.run(jupyter_height=200)

### Tabla de datos de Dash

### Red Dash AG

### Dash DAQ